In [13]:
from flask import Flask
from sqlalchemy import Column, String, Integer, ForeignKey, DateTime
from sqlalchemy.orm import relationship
from flask_sqlalchemy import SQLAlchemy

# make app and db setup

In [14]:
from models import *

In [15]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///'
db.init_app(app)

# ~~models~~ already imported

# make tables

In [16]:
with app.app_context():
  db.create_all()

# populate data

In [17]:
with app.app_context():
  import populate_db
  populate_db.populate()

# testing/querying

In [20]:
with app.app_context():
  # req = ServiceRequest(
  #     customer_id=1,
  #     service_id=1,
  #     professional_id=1,
  #     booking_date=datetime.now(),
  #     ratings=1,
  #     status='done'
  # )
  # db.session.add(req)
  # db.session.commit()
  # print(req.service.name)
  reqs = ServiceRequest.query.all()
  reqs[0].service.name

'Tap repair'